# EE-374

# Chapter 5
# The Chain 
- In the last chapter we've introduced a way for preventing double spends by enforcing periods of silence via a ticketing system that occurs at tunable frequency and its earned by mining Proof of Work
- We also required a notion of freshness from block to block to avoid tricking the system by withholding and spamming blocks at will

### 5.1 The Target
- Now that we've established the PoW inequality (as a moderately hard version of the expentially hard hash preimage problem) we'd like to estimate what the *difficulty param* $T$ should be to have $\Delta$ spacing between ticket/block to ticket/block
    - *Random oracle assumption* - Since the distribution of the image set of hash functions are not always uniform, for PoW we'd like to demand that they are uniformly distributed in $\{0,1\}^\kappa$
    - Guessing the right $\text{ctr}$ (in $B=s\|\vec{x}\|\text{ctr}$) for wining PoW has a probability of success of: $p=P[H(B)\leq T]=\frac{T}{2^\kappa}$
    - Lets assume that: *(i)* a node with typical computational power can process $q\in\mathbb{N}$ computations of $H$ per unit of time. *(ii)* There is $n\in\mathbb{N}$ computers in the network out of which the adversary controls $t\in\mathbb{N}$ (a more powerful computer can be thought as a collection of smaller computers with multiplicative power of $q\times\text{some-int-number}$)
- The expected number of queries until a successful query is $\frac{1}{p}=\frac{2^\kappa}{T}$, if the total number of queries that can be executed in the network is $nq$ then the expected block generation time is $\frac{1}{pnq}=\frac{2^\kappa}{Tnq}$, we require this to be greater than network delay: $\Delta<\frac{2^\kappa}{Tnq}\Rightarrow T<\frac{2^\kappa}{\Delta nq}$
    - The more computational power there is on the network the more we have to increase mining difficulty, and same applies if we want to delay the network paramenter
    - Lets define the *honest block production rate* $f$ and the expected time between two blocks being generated: $\eta=\frac{1}{f}$
        - *static difficulty model* - For now we can make use of a static/fixed $f\simeq (n-t)qp$

### 5.2 The Arrow of Time
- The chain structure is a consecuence of our intention of forcing *freshness* so that blocks from the past can't be saved and used at will. Turns out that the chain structure has more interesting properties that we'll discuss here
    - First lets introduce some notation, a chain $\mathcal{C}$ is a sequence of $n$ blocks $\mathcal{C}=(B_0,B_1,\ldots,B_{n-1})$ which has length $\left|\mathcal{C}\right|$ & is enumerated like arrays in Python starting at zero and going up to $n-1$
        - Like Python, the first **Genesis Block** is $\mathcal{G}=\mathcal{C}[0]$ & the tip is $\mathcal{C}[-1]=\mathcal{C}[|\mathcal{C}|-1]$
        - A chunk of the chain from the $i$-th (incluse) to the $j$-th (exclusive) element is $\mathcal{C}[i:j]=(B_i,\ldots,B_{j-1})$ 
- Now lets elaborate on how a block is *validated* and which other security notions against adversarial attacks are needed
    - Analogously to *simple ideas* do not work (Sec.4.3) for avoiding double spending in transactions, simple ideas for validating blocks also don't work eg. the idea of *"A fresh block  must extend the most recent block we’ve seen. If we receive a fresh block, we accept it. Otherwise we reject it as unfresh"* will easily fail, creating blocktrees
    - *Blocktrees* - An adversary can still create split truth scenarios eg. from Fig.5.1 party-5 could hold a block (red) just enoguh time $t_A$ before a new party-6 broadcasts his (actually legitimate) next freshest block. Then if party-7 receives the red block (paty-5's) before party-6's block at $t_B$ (because $t_B-t_A=\Delta$, theres enough spacing to traverse the network) he would split the chain, and even worse yet reject party-6's block afterwards (Fig.5.2)
    
    <img src="images/ch052-blocktree.png" width="60%">
    
- We can use the *lenght of a chain* as a reference parameter because its correlated to the time it took to produce it
    - **Longest Chain Rule**. Among the all chains on the network adopt the longest chain (the one w/ most blocks) as the *canonical chain*. If there are multiple competing chains w/ the same length choose any chain arbitrarily
    - What we have now is a worse but will lead us in the right direction. Summarized, the steps that an honest party follows are: *1)* Maintain a local canonical chain $\mathcal{C}$. *(2)* keep mining on that chain. *(3)* if mining is successful, braodcast the new block and update the local canonical chain. *(4)* if not keep mining. *(5)* in the meantime check the length of any new incoming chain, if its longer then *reorg* and **roll back** blocks as needed
    - Lets introduce some notation, $B.\vec{x}$ is the sequence of transactions component of $B=s\|\vec{x}\|\text{ctr}$. We say that $B\in\mathcal{C}$ belonging to a particular chain history and the Ledger set is initialized as empty $[]$
    - Recall the *read* functionality of a Ledger, we can now construct the basic algo (incomplete and naive until Chap.6 but workable)

<div style="background-color:rgba(0, 0, 0, 0.0470588); padding:10px 0;font-family:monospace; font-family:monospace">
<font color = "gray"># <strong>Algorithm 12</strong> The naive <i>read</i> rule</font><br>
<strong>function</strong> READ$(\mathcal{C})$<br>
&nbsp;&nbsp;$L\leftarrow []$<br>
&nbsp;&nbsp;<strong>for</strong> $B\in\mathcal{C}$ <strong>do</strong><br>
&nbsp;&nbsp;&nbsp;&nbsp;<strong>for</strong> $\text{tx}\in B.\vec{x}$ <strong>do</strong><br>
&nbsp;&nbsp;&nbsp;&nbsp;<strong>$L\leftarrow L\| \text{tx}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;<strong>end for</strong><br>
&nbsp;&nbsp;<strong>end for</strong><br>
&nbsp;&nbsp;<strong>return</strong> $L$<br>
<strong>end function</strong>
</div>


### 5.3 The Stochastic Nature of Work